### Library Import

In [1]:
import os
from typing import List, Dict
from tqdm import tqdm
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, roc_auc_score
import lightgbm as lgb
from plotly.subplots import make_subplots
import plotly.graph_objects as go


### Data Load

In [2]:
# 파일 호출
data_path: str = "/data/ephemeral/home/BTC/data"
train_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "train.csv")).assign(_type="train") # train 에는 _type = train 
test_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")).assign(_type="test") # test 에는 _type = test
submission_df: pd.DataFrame = pd.read_csv(os.path.join(data_path, "test.csv")) # ID, target 열만 가진 데이터 미리 호출
df: pd.DataFrame = pd.concat([train_df, test_df], axis=0)

In [3]:
# HOURLY_ 로 시작하는 .csv 파일 이름을 file_names 에 할딩
file_names: List[str] = [
    f for f in os.listdir(data_path) if f.startswith("HOURLY_") and f.endswith(".csv")
]

# 파일명 : 데이터프레임으로 딕셔너리 형태로 저장
file_dict: Dict[str, pd.DataFrame] = {
    f.replace(".csv", ""): pd.read_csv(os.path.join(data_path, f)) for f in file_names
}

for _file_name, _df in tqdm(file_dict.items()):
    # 열 이름 중복 방지를 위해 {_file_name.lower()}_{col.lower()}로 변경, datetime 열을 ID로 변경
    _rename_rule = {
        col: f"{_file_name.lower()}_{col.lower()}" if col != "datetime" else "ID"
        for col in _df.columns
    }
    _df = _df.rename(_rename_rule, axis=1)
    df = df.merge(_df, on="ID", how="left")


100%|██████████| 107/107 [00:03<00:00, 31.68it/s]


### EDA (Explanatory Data Analysis)

### Feature engineering

In [4]:
from data_preprocessing import *

In [5]:
# 모델에 사용할 컬럼, 컬럼의 rename rule을 미리 할당함
cols_dict: Dict[str, str] = {
    "ID": "ID",
    "target": "target",
    "_type": "_type",
    "hourly_market-data_price-ohlcv_all_exchange_spot_btc_usd_close": "close",
    "hourly_market-data_open-interest_all_exchange_all_symbol_open_interest": "open_interest",
    "hourly_network-data_difficulty_difficulty": "difficulty",
    "hourly_network-data_supply_supply_total": "supply_total",
    "hourly_network-data_utxo-count_utxo_count": "utxo_count"
}
df = df[cols_dict.keys()].rename(cols_dict, axis=1)
df.shape

(11552, 8)

In [6]:
# continuous 열을 따로 할당해둠
conti_cols: List[str] = [
    "close",
    "open_interest",
    "difficulty",
    "supply_total",
    "utxo_count"
]

# 최대 24시간의 shift 피쳐를 계산
shift_list = shift_feature(
    df=df, conti_cols=conti_cols, intervals=[_ for _ in range(1, 24)]
)

# concat 하여 df 에 할당
df = pd.concat([df, pd.concat(shift_list, axis=1)], axis=1)

In [7]:
# _type에 따라 train, test 분리
train_df = df.loc[df["_type"]=="train"].drop(columns=["_type"])
test_df = df.loc[df["_type"]=="test"].drop(columns=["_type"])

## Model Training
### LightGBM + GridsearchCV

In [8]:
X_train = train_df.drop(["ID", "target", "close"], axis=1)
y_train = train_df["close"]
X_test = test_df.drop(["ID", "target", "close"], axis=1)
y_test = test_df["close"]
target = train_df["target"]

In [34]:
from sklearn.model_selection import GridSearchCV, KFold

# lgb params
params = {
    "boosting_type": "gbdt",
    "metric": "mean_squared_error",
    "verbose": -1,
    #"device": "gpu",
}

# 파라미터 그리드 설정(grid search할 하이퍼파라미터후보)
# param_grid = {
#     "num_leaves": [31, 50, 100],
#     "learning_rate": [0.01, 0.05, 0.1],
#     "n_estimators": [30, 50, 100],
# }

param_grid = {
    "num_leaves": [30],
    "learning_rate": [0.05],
    "n_estimators": [50],
}

# LightGBM 모델 설정
lgb_model = lgb.LGBMRegressor(**params)

# GridSearchCV 설정
grid_search = GridSearchCV(
    estimator=lgb_model,
    param_grid=param_grid,
    scoring='neg_mean_squared_error',
    n_jobs=-1,
    verbose=1
)

# 모델 학습 (튜닝 시작)
grid_search.fit(X_train, y_train)

# 최적의 모델 선택
best_lgb_model = grid_search.best_estimator_
print("Best Hyperparameters: ", grid_search.best_params_)


Fitting 5 folds for each of 1 candidates, totalling 5 fits
Best Hyperparameters:  {'learning_rate': 0.05, 'n_estimators': 50, 'num_leaves': 20}


In [35]:
def close_to_class(series: pd.Series) -> pd.Series:
    """close 변수를 target값으로 변환하는 함수입니다.

    Args:
        series (pd.Series): 변환을 원하는 close 변수

    Returns:
        pd.Series: 변환된 target 값
    """
    close = pd.DataFrame()
    close['close'] = series
    close['close_lag1'] = close['close'].shift(1)
    close['close_lag1_percent'] = (close['close'] - close['close_lag1']) / close['close_lag1']
    close['class'] = close['close']
    for i in range(close.shape[0]):
        if close.iloc[i]['close_lag1_percent'] < -0.005:
            close.iloc[i, close.columns.get_loc('class')] = 0
        elif close.iloc[i]['close_lag1_percent'] < 0:
            close.iloc[i, close.columns.get_loc('class')] = 1
        elif close.iloc[i]['close_lag1_percent'] < 0.005:
            close.iloc[i, close.columns.get_loc('class')] = 2
        else:
            close.iloc[i, close.columns.get_loc('class')] = 3
            
    return close['class']

In [36]:
from sklearn.metrics import mean_absolute_error, mean_squared_error, mean_absolute_percentage_error, r2_score
import warnings
warnings.filterwarnings("ignore")

def evaluate(valid_target, y_valid, y_pred, metric):
    if metric == 'accuracy':
        classes_pred = close_to_class(y_pred)
        return accuracy_score(valid_target, classes_pred)
    if metric == 'mae':
        return mean_absolute_error(y_valid, y_pred)
    if metric == "mse":
        return mean_squared_error(y_valid, y_pred)
    if metric == "mape":
        mae = mean_absolute_error(y_valid, y_pred)
        return mae / np.mean(y_valid)

In [37]:
def model_train(model, X_train, y_train, cv, metric):
    #kfold = KFold(n_splits=cv, shuffle=True, random_state=42)
    kfold = KFold(n_splits=cv)
    score_list = []
    for train_index, valid_index in kfold.split(X_train):
        X_train_fold, y_train_fold = X_train.iloc[train_index], y_train.iloc[train_index]
        X_valid, y_valid = X_train.iloc[valid_index], y_train.iloc[valid_index]
        
        vaild_target = target.iloc[valid_index]

        # preprocessing
        X_train_fold.fillna(X_train_fold.mean(), inplace=True)
        y_train_fold.fillna(y_train_fold.mean(), inplace=True) 
        X_valid.fillna(X_valid.mean(), inplace=True)
        y_valid.fillna(y_valid.mean(), inplace=True)  # 이 부분을 mice와 같은 방법으로 조정할 예정. feature selection 등도 여기에서.

        #model.fit(X_train_fold.drop("target", axis=1), y_train_fold)
        model.fit(X_train_fold, y_train_fold)
        # y_pred = model.predict(X_valid.drop("target", axis=1))
        y_pred = model.predict(X_valid)
        score = evaluate(vaild_target, y_valid, y_pred, metric=metric)
        score_list.append(score)

    return np.mean(score_list)

In [38]:
avg = model_train(best_lgb_model, X_train, y_train, cv=5, metric="accuracy")

print("LGB model accuracy:", avg)

LGB model accuracy: 0.38687214611872145


### Inference

In [30]:
X_test = test_df.drop(["ID", "target", "close"], axis=1)
y_test = test_df["close"]

X_test.fillna(X_test.mean(), inplace=True)
y_test.fillna(y_test.mean(), inplace=True)

In [31]:
# lgb predict
y_test_pred = best_lgb_model.predict(X_test)
y_test_pred_class = close_to_class(y_test_pred)

### Output File Save

In [33]:
# output file 할당후 save 
submission_df = submission_df.assign(target = y_test_pred_class)
submission_df["target"] = submission_df["target"].astype(np.int8)
submission_df.to_csv("output.csv", index=False)